<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: COVID-19 Dashboard
23 de Fevereiro de 2024.<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)<br>
Aluno: [Raul Lemelle](https://www.linkedin.com/in/raul-lemelle)

---


# **COVID Dashboard**

## 1\. Contexto

### **1.1. TLDR**

 - **Dashboard**:
  - Google Data Studio ([link](https://lookerstudio.google.com/reporting/f656e94c-5571-43bb-bf39-fd7f469e4656)).
 - **Processamento**:
  - Kaggle Notebook ([link](https://www.kaggle.com/code/raullemelle/covid-19-dashboard)).
 - **Fontes**:
  - Casos pela universidade John Hopkins ([link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports));
  - Vacinação pela universidade de Oxford ([link](https://covid.ourworldindata.org/data/owid-covid-data.csv)).

O desenvolvimento do Dashboard de Casos de COVID-19 abrange a coleta e processamento de dados atualizados por meio de APIs ou fontes confiáveis, seguido pela escolha de uma linguagem de programação, como Python, e um framework de visualização, por exemplo o Google Data Studio.

O código resultante estabelece uma base de dados consistente para representar de forma clara o número de infectados, mortos e métricas estatísticas relevantes.

O Dashboard é cuidadosamente projetado para ser responsivo, proporcionando uma experiência otimizada, e oferece opções de personalização, permitindo aos usuários selecionarem regiões específicas e intervalos de tempo.

A atualização automática dos dados, juntamente com testes extensivos e a incorporação de recursos adicionais, como filtros avançados, contribuem significativamente para a robustez e utilidade do Dashboard.

Após o desenvolvimento, o Dashboard pode ser hospedado online, garantindo um acesso fácil e compreensível ao público, proporcionando informações detalhadas sobre a situação da COVID-19.

### **1.2. Dados**


Os dados relativos aos casos de COVID-19 são compilados pelo Centro de Ciência de Sistemas e Engenharia da renomada Universidade **John Hopkins** ([link](https://www.jhu.edu)). Essas informações são atualizadas diariamente desde janeiro de 2020, oferecendo uma análise temporal detalhada em níveis geográficos que incluem países, estados e condados. Para acessar o projeto, visite o website disponível [aqui](https://systems.jhu.edu/research/public-health/ncov/), enquanto os dados processados podem ser obtidos neste [link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports).<br>
A seguir, apresentamos os resultados do processamento desses dados.

 - **date**: data de referência;
 - **state**: estado;
 - **country**: país;
 - **population**: população estimada;
 - **confirmed**: número acumulado de infectados;
 - **confirmed_1d**: número diário de infectados;
 - **confirmed_moving_avg_7d**: média móvel de 7 dias do número diário de infectados;
 - **confirmed_moving_avg_7d_rate_14d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **deaths**: número acumulado de mortos;
 - **deaths_1d**: número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias do número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **month**: mês de referência;
 - **year**: ano de referência.


Os dados relativos à vacinação contra a COVID-19 são consolidados pelo projeto Nosso Mundo em Dados (Our World in Data ou OWID) da renomada Universidade de **Oxford** ([link](https://www.ox.ac.uk)). Essas informações são atualizadas diariamente desde janeiro de 2020, proporcionando uma análise temporal detalhada em níveis geográficos abrangendo países. Para acessar o projeto, visite o website disponível [aqui](https://ourworldindata.org), enquanto os dados podem ser obtidos neste [link](https://covid.ourworldindata.org/data/owid-covid-data.csv).<br>
A seguir, apresentamos os resultados derivados do processamento desses dados.

 - **date**: data de referência;
 - **country**: país;
 - **population**: população estimada;
 - **total**: número acumulado de doses administradas;
 - **one_shot**: número acumulado de pessoas com uma dose;
 - **one_shot_perc**: número acumulado relativo de pessoas com uma dose;
 - **two_shots**: número acumulado de pessoas com duas doses;
 - **two_shot_perc**: número acumulado relativo de pessoas com duas doses;
 - **three_shots**: número acumulado de pessoas com três doses;
 - **three_shot_perc**: número acumulado relativo de pessoas com três doses;
 - **month**: mês de referência;
 - **year**: ano de referência.

## 2\. Pacotes e bibliotecas

In [1]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

## 3\. Casos

### **3.1. Extração**

Dados de Casos da universidade Jonh Hopkins.

In [2]:
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',')

In [3]:
cases.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-13 05:22:15,33.93911,67.709953,53584,2301,44608,6675,Afghanistan,137.647787,4.294192
1,NaN,NaN,NaN,Albania,2021-01-13 05:22:15,41.15330,20.168300,64627,1252,38421,24954,Albania,2245.708527,1.937271
2,NaN,NaN,NaN,Algeria,2021-01-13 05:22:15,28.03390,1.659600,102641,2816,69608,30217,Algeria,234.067409,2.743543
3,NaN,NaN,NaN,Andorra,2021-01-13 05:22:15,42.50630,1.521800,8682,86,7930,666,Andorra,11236.653077,0.990555
4,NaN,NaN,NaN,Angola,2021-01-13 05:22:15,-11.20270,17.873900,18343,422,15512,2409,Angola,55.811022,2.300605


Nesse sentido, será necessário realizar iterações dentro de um intervalo de tempo predefinido para efetuar a extração.

In [4]:
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)

In [5]:
start_date = datetime(2021,  1,  1)
end_date   = datetime(2021, 12, 31)

De forma iterativa, optaremos por selecionar as colunas de interesse, assim como as linhas pertinentes ao Brasil.

In [6]:
cases = None
cases_is_empty = True

for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  case = pd.read_csv(data_source_url, sep=',')

  case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
  case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
  case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))

  if cases_is_empty:
    cases = case
    cases_is_empty = False
  else:
    cases = cases.append(case, ignore_index=True)

<ipython-input-6-24f56788c23e>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cases = cases.append(case, ignore_index=True)
<ipython-input-6-24f56788c23e>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cases = cases.append(case, ignore_index=True)
<ipython-input-6-24f56788c23e>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cases = cases.append(case, ignore_index=True)
<ipython-input-6-24f56788c23e>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cases = cases.append(case, ignore_index=True)
<ipython-input-6-24f56788c23e>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future vers

In [7]:
cases.query('Province_State == "Sao Paulo"').head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
24,Sao Paulo,Brazil,1466191,46775,3192.990778,2021-01-01
51,Sao Paulo,Brazil,1467953,46808,3196.827966,2021-01-02
78,Sao Paulo,Brazil,1471422,46845,3204.382565,2021-01-03
105,Sao Paulo,Brazil,1473670,46888,3209.278136,2021-01-04
132,Sao Paulo,Brazil,1486551,47222,3237.329676,2021-01-05


### **3.2. Wrangling**

Vamos agora manipular os dados para o dashboard, priorizando assegurar uma granularidade consistente e aprimorar a qualidade da base de dados.

In [8]:
cases.head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
0,Acre,Brazil,41689,796,4726.992352,2021-01-01
1,Alagoas,Brazil,105091,2496,3148.928928,2021-01-01
2,Amapa,Brazil,68361,926,8083.066602,2021-01-01
3,Amazonas,Brazil,201574,5295,4863.536793,2021-01-01
4,Bahia,Brazil,494684,9159,3326.039611,2021-01-01


In [9]:
cases.shape

(9828, 6)

In [10]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9828 entries, 0 to 9827
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province_State  9828 non-null   object        
 1   Country_Region  9828 non-null   object        
 2   Confirmed       9828 non-null   int64         
 3   Deaths          9828 non-null   int64         
 4   Incident_Rate   9828 non-null   float64       
 5   Date            9828 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 460.8+ KB


Nome das Colunas.

In [11]:
cases = cases.rename(
  columns={
    'Province_State': 'state',
    'Country_Region': 'country'
  }
)

for col in cases.columns:
  cases = cases.rename(columns={col: col.lower()})

Nome dos estados

In [12]:
states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

Enriquecimento dos dados com novas colunas

Chaves temporais:

In [13]:
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year']  = cases['date'].apply(lambda date: date.strftime('%Y'))

População estimada do estado:

In [14]:
cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)

Número, média móvel (7 dias) e estabilidade (14 dias) de casos e mortes por estado:

In [15]:
cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate < 0.75:
    status = 'downward'
  elif rate > 1.15:
    status = 'upward'
  else:
    status = 'stable'

  return status


for state in cases['state'].drop_duplicates():

  cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values(by=['date'])

  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
  cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

  if cases_is_empty:
    cases_ = cases_per_state
    cases_is_empty = False
  else:
    cases_ = cases_.append(cases_per_state, ignore_index=True)

cases = cases_
cases_ = None

<ipython-input-15-3b759f7b3fe1>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cases_ = cases_.append(cases_per_state, ignore_index=True)
<ipython-input-15-3b759f7b3fe1>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cases_ = cases_.append(cases_per_state, ignore_index=True)
<ipython-input-15-3b759f7b3fe1>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cases_ = cases_.append(cases_per_state, ignore_index=True)
<ipython-input-15-3b759f7b3fe1>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cases_ = cases_.append(cases_per_state, ignore_index=True)
<ipython-input-15-3b759f7b3fe1>:38: FutureWarning: The frame.append method is de

Garantir o tipo do dado é fundamental para consistência da base de dados. Vamos fazer o *type casting* das colunas.

In [16]:
cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

 Reorganizar as colunas.

In [17]:
cases = cases[[
    'date', 'country', 'state', 'population',
    'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d',
    'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend',
    'deaths', 'deaths_1d', 'deaths_moving_avg_7d',
    'deaths_moving_avg_7d_rate_14d', 'deaths_trend',
    'month', 'year'
]]

### **3.3. Carregamento**

Com os dados manipulados, procederemos à sua persistência em disco, realizaremos o download e, posteriormente, o carregaremos no Google Data Studio.

In [18]:
cases.to_csv('./covid-cases.csv', sep=',', index=False)

## 4\. Vacinação

### **4.1. Extração**

Dados de vacinação da universidade Oxford.

In [19]:
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

<ipython-input-19-608e6d2dd56d>:1: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)


In [20]:
vaccines.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-08,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-09,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


Selecionar as colunas e linhas referentes ao Brasil.

In [21]:
vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

In [22]:
vaccines.head()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-05
1,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-06
2,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-07
3,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-08
4,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-09


### **4.2. Wrangling**

Vamos realizar a manipulação dos dados para o dashboard, priorizando a obtenção de uma granularidade consistente e aprimoramento da qualidade da base de dados.

In [23]:
vaccines.head()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-05
1,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-06
2,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-07
3,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-08
4,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-09


In [24]:
vaccines.shape

(1492, 7)

In [25]:
vaccines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   location                 1492 non-null   object        
 1   population               1492 non-null   float64       
 2   total_vaccinations       695 non-null    float64       
 3   people_vaccinated        691 non-null    float64       
 4   people_fully_vaccinated  675 non-null    float64       
 5   total_boosters           455 non-null    float64       
 6   date                     1492 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 81.7+ KB


Para lidar com dados faltantes, adotaremos a estratégia de preenchê-los com o valor válido mais próximo anterior.

In [26]:
vaccines = vaccines.fillna(method='ffill')


Vamos também filtrar a base de dados de acordo com a coluna "date" para assegurar que ambas as bases de dados abranjam o mesmo período temporal.

In [27]:
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

Alteração do nome das colunas.

In [28]:
vaccines = vaccines.rename(
  columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
  }
)

Enriquecimento dos dados com novas colunas.

Chaves temporais:

In [29]:
vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))

Dados relativos:

In [30]:
vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

Garantir o tipo do dado é fundamental para consistência da base de dados. Vamos fazer o type casting das colunas.

In [31]:
vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

Reorganizar as colunas.

In [32]:
vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot',
                     'one_shot_perc', 'two_shots', 'two_shots_perc',
                     'three_shots', 'three_shots_perc', 'month', 'year'
]]

### **4.3. Carregamento**

Com os dados manipulados, procederemos à sua persistência em disco, realizaremos o download e, posteriormente, o carregaremos no Google Data Studio.

In [33]:
vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)